In [136]:
print('ping')

ping


In [137]:
import os
import glob
import numpy as np
import multiprocessing

khome = '/global/scratch/nathan_sandford/kurucz_old'
outputdir = khome+'/synthetic_spectra'
outputfile = 'CR_spectra_t3200_g0.2_fehm0.0_vturb2.0_2.npz'

'''
Set length of wavelength
wavelength_length = 537529 for whole wavelength
wavelength_length = 224165 for 4500 AA - 9500 AA
wavelength_length = 113848 for 6500 AA - 9500 AA
'''
wavelength_min = 2900  # Cannot be < 3000 AA
wavelength_max = 19000  # Cannot be > 18000 AA
wavelength_length = 537529

'''
Replace bad models with reference model
(necessary for CR calculations)
'''
replace= True

# List of models for which spectra have been generated
list_models = [os.path.basename(x)[5:10]
               for x in glob.glob(khome+"/grids/*/spec/*")]
list_models.sort()

# Number of synthetic spectra generated
nmodels = int(len(list_models))
print('Detected %i models' % nmodels)

# List of files containing spectra
list_spec = glob.glob(khome+"/grids/*/spec/*")
list_spec.sort()

# Array containing Solar elemental abundances from H -> Es
# H and He are fractional abundances while the rest are log10[X/H]
# Added filler 0 to line up indices with atomic #
print('Defining Solar abundance array')
elems_array_solar = np.array([0, 0.92068, 0.07837,
                             -10.99, -10.66,  -9.34,  -3.61,  -4.21,
                             -3.35,  -7.48,  -4.11,  -5.80,  -4.44,
                             -5.59,  -4.53,  -6.63,  -4.92,  -6.54,
                             -5.64,  -7.01,  -5.70,  -8.89,  -7.09,
                             -8.11,  -6.40,  -6.61,  -4.54,  -7.05,
                             -5.82,  -7.85,  -7.48,  -9.00,  -8.39,
                             -9.74,  -8.70,  -9.50,  -8.79,  -9.52,
                             -9.17,  -9.83,  -9.46, -10.58, -10.16,
                             -20.00, -10.29, -11.13, -10.47, -11.10,
                             -10.33, -11.24, -10.00, -11.03,  -9.86,
                             -10.49,  -9.80, -10.96,  -9.86, -10.94,
                             -10.46, -11.32, -10.62, -20.00, -11.08,
                             -11.52, -10.97, -11.74, -10.94, -11.56,
                             -11.12, -11.94, -11.20, -11.94, -11.19,
                             -12.16, -11.19, -11.78, -10.64, -10.66,
                             -10.42, -11.12, -10.87, -11.14, -10.29,
                             -11.39, -20.00, -20.00, -20.00, -20.00,
                             -20.00, -20.00, -12.02, -20.00, -12.58,
                             -20.00, -20.00, -20.00, -20.00, -20.00,
                             -20.00, -20.00])

# Initialize arrays for # of models and length of wavelength
print('Expecting Synthetic Spectra with %i pixels' % wavelength_length)


def read_spec(i):
    spec = list_spec[i]
    model = list_models[i]
    print('Reading in spectra from model %s' % model)
    good_model = True

    # Unzip if .gz file
    if spec[-3:] == '.gz':
        os.system("gunzip "+spec)
        spec = spec[:-3]

    try:
        # Read in spectra file
        temp = np.genfromtxt(spec)
        temp = temp.transpose()

        # Read in Wavelength, Spectrum, and Theoretical Continuum
        wavelength = temp[0][(temp[0] > wavelength_min/10)
                             & (temp[0] < wavelength_max/10)] * 10  # AA
        spectra = temp[1][(temp[0] > wavelength_min/10)
                          & (temp[0] < wavelength_max/10)]
        continuum = temp[2][(temp[0] > wavelength_min/10)
                            & (temp[0] < wavelength_max/10)]

        # Read in labels file output by generate_initial_atm.py
        tmp = np.load(khome+"/grids/"+model+"/"+model+"_labels.npz")
        teff = tmp['teff']
        logg = tmp['logg']
        v_micro = tmp['v_micro']
        # Add Filler 0
        elems_array_unscaled = np.concatenate([[0], tmp['elems_array']])
        tmp.close()

        # Scale abundances to solar (except H & He)
        elems_array = elems_array_unscaled
        elems_array[3:] = elems_array_unscaled[3:] - elems_array_solar[3:]

        # Select which labels to keep
        all_labels = np.concatenate([[teff], [logg], [v_micro],
                                     elems_array[3:]])

        # Catch bad model generation: incomplete wavelength range
        if len(wavelength) != wavelength_length:
            print('Incomplete wavelength coverage, got %i pixels but expected %i'
                  % (len(wavelength), wavelength_length))
            print('could not load model %s' % model)
            wavelength = np.empty(wavelength_length)
            spectra = np.empty(wavelength_length)
            continuum = np.empty(wavelength_length)
            all_labels = np.empty(len(elems_array_solar[3:])+3)
            good_model = False

    except:
        # Catch bad model generation
        print('Something went wrong, could not load model %s' % model)
        wavelength = np.empty(wavelength_length)
        spectra = np.empty(wavelength_length)
        continuum = np.empty(wavelength_length)
        all_labels = np.empty(len(elems_array_solar[3:])+3)
        good_model = False

    # Close and compress spectra file
    os.system("gzip "+spec)

    return(wavelength, spectra, continuum, all_labels, model, good_model)


print('Beginning batch reading of spectra')
pool = multiprocessing.Pool(multiprocessing.cpu_count())
temp = pool.map(read_spec, range(len(list_spec)))
temp = list(zip(*temp))
wavelength, spectra, continuum, all_labels, model, good_model = temp

if replace and np.all(good_model):
    print('No bad models found')
elif replace and not np.all(good_model):
    bad_model_ind = np.concatenate(np.argwhere(np.array(good_model) == False))
    for ind in bad_model_ind:
        spectra[ind] = spectra[0]
        continuum[ind] = continuum[0]
        print('%i bad models removed' % len(bad_model_ind)) 
else:
    # Remove bad models
    good_model_ind = np.concatenate(np.argwhere(np.array(good_model) == True))
    wavelength = [wavelength[i] for i in good_model_ind]
    spectra = [spectra[i] for i in good_model_ind]
    continuum = [continuum[i] for i in good_model_ind]
    all_labels = [all_labels[i] for i in good_model_ind]
    model = [model[i] for i in good_model_ind]
    print('%i bad models removed' % (len(list_models) - len(good_model_ind)))

# Save all spectra and labels together
np.savez(outputdir + '/' + outputfile,
         wavelength=wavelength[0],
         spectra=spectra, continuum=continuum,
         labels=all_labels, model=model)

print('Saved all synthetic spectra to %s' % (outputdir+'/'+outputfile))


Detected 202 models
Defining Solar abundance array
Expecting Synthetic Spectra with 537529 pixels
Beginning batch reading of spectra
Reading in spectra from model aaaae
Reading in spectra from model aaaaa
Reading in spectra from model aaaag
Reading in spectra from model aaaac
Reading in spectra from model aaaff
Reading in spectra from model aaahf
Reading in spectra from model aaacc
Reading in spectra from model aaadi
Reading in spectra from model aaafc
Reading in spectra from model aaagh
Reading in spectra from model aaabe
Reading in spectra from model aaagd
Reading in spectra from model aaaea
Reading in spectra from model aaagf
Reading in spectra from model aaabi
Reading in spectra from model aaahb
Reading in spectra from model aaadg
Reading in spectra from model aaagb
Reading in spectra from model aaaee
Reading in spectra from model aaahh
Reading in spectra from model aaacg
Reading in spectra from model aaaca
Reading in spectra from model aaabg
Reading in spectra from model aaaba
Rea